# **LangChain Templating: Syntax and Examples**

## **📌Overview**
Prompt templates allow for the dynamic generation of prompts by inserting variables into a structured format. \
This enables the creation of flexible and reusable prompts for different tasks.

LangChain supports multiple **templating syntaxes**, including (but not limited to):

- **Basic String Formatting (`{variable}`)**  
- **Multiple Placeholders in a Single Template (`{var1}, {var2}`)**  
- **Tuple-Based Formatting (`("system", "text {var}")`)**  
- **Using `HumanMessage` Objects (`HumanMessage(content="text")`)**  

In [ ]:
# Install dependencies 
%pip install -q python-dotenv langchain-core langchain-google-genai langchain
%pip uninstall -y google-generativeai google-ai-generativelanguage
%pip install -q google-generativeai==0.8.4 google-ai-generativelanguage==0.6.15

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

## **Obtain a Google Gemini API Key (GOOGLE COLLAB SETUP):**

If you have a Google Gemini API Key: 
- Copy your API key and replace "your_google_api_key_here" in the code below

Otherwise:  
- Go to the Google AI Studio API Console: [Google AI Studio](https://aistudio.google.com/prompts/new_chat)
- Sign in with your Google account and create a new API key.
- Copy your API key and replace "your_google_api_key_here" in the code below

In [ ]:
# Set your Google API key manually
import os
os.environ["GOOGLE_API_KEY"] = "your_google_api_key_here"

## **Load Environment Variables (LOCAL SETUP)**

In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

---

## **Basic String Formatting**  

Uses LangChain's built-in `{}` placeholders to dynamically insert values into prompts.  

In [ ]:
# Basic string 
template = "Tell me an interesting fact about {topic}"
prompt_template = ChatPromptTemplate.from_template(template)

prompt = prompt_template.invoke({"topic": "black holes"})
print(prompt)

## **Multiple Placeholders in a Single Template**  

Supports multiple dynamic placeholders within a single prompt template.  

In [ ]:
# Multiple placeholders
template_multiple = """You are a convincing essay writer.
Human: Write a {essay_length} word essay on {essay_position}.
Writer: """
prompt_template = ChatPromptTemplate.from_template(template_multiple)

prompt = prompt_template.invoke({
    "essay_length": 200, 
    "essay_position": "Gorillas are a reasonable pet to have in the household"})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

## **Tuple-Based Formatting**
Uses tuples to format system and human messages dynamically in multi-turn conversations.

In [ ]:
# Tuple-based formatting
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world travel expert."
            "Provide concise, actionable insights on travel experiences."),
    ("human", "What are the top {activity} experiences in {destination}?"),
])

prompt = prompt_template.invoke({"destination": "Tokyo", "activity": "food"})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

messages=[SystemMessage(content='You are a world travel expertProvide concise, actionable insights on travel experiences.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are the top food experiences in Tokyo?', additional_kwargs={}, response_metadata={})]
**High-End:**  Sushi Dai (Tsukiji, expect a queue),  Ginza Sushiko (classic, pricey).

**Mid-Range:**  Ramen at Ichiran (customizable bowls),  Monjayaki in Tsukishima (savory pancake).

**Unique:**  Robot Restaurant (show & dinner, kitsch),  standing sushi bars (conveyor belt, affordable).

**Street Food:**  Takoyaki (octopus balls),  yakitori (grilled skewers),  crepes.


**Actionable Tip:**  Make reservations for popular restaurants, especially sushi. Explore diverse neighborhoods for authentic experiences.


## **Importance of System and Human Messages**
Using system messages allows override of LLMs safety filters (to extent) and default behaviours 

In [ ]:
# Multiple placeholders
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a bot that RESPONDS ONLY in UPPERCASE."),
    ("human",  "Hello, how are you today?"),
])

prompt = prompt_template.invoke({})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

# -------------------------------------------------------

# Tuple-based formatting
prompt_template = ChatPromptTemplate.from_messages(
    """You are a bot that RESPONDS ONLY in UPPERCASE.
    Human: Hello, how are you today?
    Bot:
    """
)
prompt = prompt_template.invoke({})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)


## **Using HumanMessage Objects**
Allows structured human input messages within a LangChain chat prompt.

In [14]:
# HumanMessage objects
messages = [
    ("system", "You are a tour guide."),
    HumanMessage(content="Describe three must-visit places."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)

prompt = prompt_template.invoke({"location": "Halifax"})
print(prompt)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

messages=[SystemMessage(content='You are a tour guide.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Describe three must-visit places.', additional_kwargs={}, response_metadata={})]
Alright folks, gather 'round!  For those of you looking for unforgettable experiences, I've got three must-visit places that offer something truly unique:

1. **Machu Picchu, Peru:**  For the history buff and adventure seeker alike, Machu Picchu is unparalleled.  Imagine trekking through lush cloud forests, the air thick with the scent of orchids, before emerging onto a sun-drenched mountaintop to witness this breathtaking Inca citadel.  The sheer scale of the stonework, the stunning mountain views, the palpable sense of history – it's an experience that will stay with you forever.  Be warned, it's not for the faint of heart – the trek can be challenging, but the reward is immeasurable.

2. **The Great Barrier Reef, Australia:** Prepare to be awestruck by the vibrant kaleidoscope of l

---

## **Templating Example**

In [12]:
# Extended example of how to start using templating 

# Instantiate LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Define arguments and input selections
args_dictionary = {"field": "", "concept": "", "difficulty": ""}
valid_difficulties = {"basic", "intermediate", "advanced"}

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are an expert tutor. Your role is to explain concepts clearly for a given field: 
        - If relevant to the field, give a clear explanation. 
        - Otherwise, respond: "I'm sorry, but I cannot help with that."""),
    ("human", "Explain {concept} in {difficulty} terms."),    
    ("ai", "Sure! Here's a breakdown of {concept} at the {difficulty} level.")
])

# Get user inputs
field = input("What field do you want to learn about? ").strip()
while not field: 
    print("!Error!: Field cannot be empty.")
    field = input("What field do you want to learn about? ").strip()

concept = input("What concept would you like to learn about: ").strip()
while not concept:
    print("!Error!: Field cannot be empty.")
    concept = input("What concept would you like to learn about: ").strip()

difficulty = input(f"What depth of understanding would you like to learn \
                   about {concept}? (basic, intermediate, advanced): ").strip().lower()
while difficulty not in valid_difficulties: 
    print("Error: Please enter a valid difficulty level (basic, intermediate, advanced).")
    difficulty = input(f"What depth of understanding would you like to learn \
                   about {concept}? (basic, intermediate, advanced): ").strip().lower()
    
print("Input Accepted!")
print(f"- Field: {field}")
print(f"- Concept: {concept}")
print(f"- Difficulty: {difficulty}\n")

# Update dictionary with user input
args_dictionary.update({
    "field": field,
    "concept": concept,
    "difficulty": difficulty
})

# Format prompt  
prompt = prompt_template.invoke(args_dictionary)

# Get LLM response
response = llm.invoke(prompt)
print(response.content)

Error: Please enter a valid difficulty level (basic, intermediate, advanced).
Input Accepted!
- Field: math
- Concept: quantum state
- Difficulty: advanced

A quantum state is a complete description of a quantum system.  Unlike classical mechanics where a system's state is defined by specifying values for all its observable properties (like position and momentum), a quantum state is fundamentally probabilistic.  It doesn't tell us the *definite* values of observables, but rather the *probability amplitudes* for obtaining different measurement outcomes.  This is encapsulated mathematically in a few key ways:

**1. Hilbert Space Representation:**  The most common and rigorous representation is through vectors in a complex Hilbert space.  This space is a vector space with an inner product that allows for the calculation of probability amplitudes.

* **Ket Notation:**  A quantum state is represented by a ket vector, denoted as |ψ⟩ (pronounced "ket psi").  This vector lives in the Hilbert s